# Genesis Effect Simulation
This is a simulation of the Genesis Effect using the particle system method described by William Reeves in his 1983 
paper "[Particle systems - a technique for modeling a class of fuzzy objects](https://onesearch.library.uwa.edu.au/discovery/fulldisplay?docid=crossref10.1145/964967.801167&context=PC&vid=61UWA_INST:UWA&search_scope=MyInst_and_CI&tab=Everything&lang=en "UWA OneSearch")". This paper outlines a particle system for simulating the Genesis effect in the film *Star Trek 2: The Wrath of Khan*. This application of particle systems was revolutionary, and represented a huge leap in computer graphics simulations.

We have implemented the particle system described by Reeves in an attempt to replicate his results. This particle system includes the following:
- A number of particle attributes
  - position, velocity and acceleration
  - initial and final color, with interpolation
  - lifespan
- A heirachical system, in which particle systems spawn particles which are themselves particle systems

All reference material came from the original paper. With additional non technical information found on [Khan Academy](https://www.khanacademy.org/partner-content/pixar/effects/particle/pi/genesis-effect "Khan Academy").

##### Library Use and Imports
We have used the python graphics library and OpenGL interface [Pyglet](https://pyglet.readthedocs.io/en/pyglet-1.3-maintenance/index.html# "Pyglet Homepage") for all of our drawing and rendering requirements.

Pyglet can be installed with Anaconda or Pip with either:

```$> conda install pyglet```

or

```$> pip install pyglet```

In [1]:
# import libraries
import math, random
import numpy as np

# graphics library
import pyglet
from pyglet.gl import *

### Class Definitions
We have defined two classes for this particle system. `Particle` which represents a bottom level, rendered point on the screen, and `Particle System` which is responsible for spawning `Particle` instances.

Note that these classes are highly similar in structure to those used in the fireworks simulation attached to this notebook. This is due to the same underlying particle system being used for both systems (with very minor modifications).

##### Constants
These constants are defined and used throughout the classes as either constant values or default parameters

In [2]:
# constants/default values
RADIUS = 25 # planet radius
GRAVITY = -50 # gravity magnitude in 'units per frame per frame'

P_VEL, P_VEL_VAR = 15, 20 # mean and variance of particle velocity MAGNITUDE in 'units per frame'
P_VAR = 5 # variance of individal components of velocity (effects ejection angle)
SPAWN_RAD = 0.1 # spawn area variation (polar/azimuth in spherical coords.) <- this is an angle in radians

CHILDREN = 15 # maximum number of children to spawn per update

##### Particle System Class
Contains a constructor `__init__`, and two methods: `update` and `spawn`, which update the system and spawn children into it.
    
This class is highly simmilar to its counterpart employed in the fireworks simulation. Additional mathematics was required to calculate vectors in 3D space, however the class is structurally and functionally identical.

In [3]:
class ParticleSystem:
    def __init__(
        self, position,
        startColor, endColor,
        lifespan=0.75, numChildren=CHILDREN,
        speed=P_VEL, speedVariance=P_VEL_VAR, var=P_VAR,
        spawnrad=SPAWN_RAD
    ):
        # define position/vector properties
        self.spos = xyzToSpherical(*position) # spherical representation of position
        self.pos = position # cartesian replresentation of position
        self.normal = normalise(self.pos) # vector normal to surface
        self.gravity = [GRAVITY*self.normal[0], GRAVITY*self.normal[1], GRAVITY*self.normal[2]] # gravity vector
        self.speed, self.speedV = speed, speedVariance # mean particle ejection magnitude and variance
        self.var=var # variance of particle ejection direction
        self.spawnrad = spawnrad # angular offset (radians) around particle system in which particles will be spawned from
        # define color properties
        self.col = startColor
        self.colS, self.colF = startColor, endColor
        # define particle spawning properties
        self.children = []
        self.spawnNum = numChildren
        self.canSpawn = True
        # define life or death properties
        self.age = 0
        self.lifespan = lifespan
        self.alive = True

    # updates the particle systems internal properties
    # parameter 'dt' is the (simulated) duration in seconds since last update
    def update(self, dt):
        # update age
        self.age += dt
        # update children
        for index, child in reversed(list(enumerate(self.children))):
            child.update(dt)
            if not child.alive:
                del self.children[index]
        # spawn children
        if self.canSpawn: self.spawn()
        # update color
        self.col = colorInterp(self.colS, self.colF, self.age, self.lifespan)
        # kill if too old
        self.alive = self.age < self.lifespan

    # spawns particles into the simulation
    def spawn(self):
        for count in range(15):
            # define launch position of particle
            pos = sphericalToXYZ(self.spos[0],
                                 self.spos[1]+random.uniform(-self.spawnrad, self.spawnrad),
                                 self.spos[2]+random.uniform(-self.spawnrad, self.spawnrad))
            # determine the magnitude of particles velocity
            mag = self.speed + random.uniform(-self.speedV, self.speedV)
            # calculate the velocity vector of the particle
            vel = [mag*self.normal[0]+random.uniform(-self.var, self.var),
                    mag*self.normal[1]+random.uniform(-self.var, self.var),
                    mag*self.normal[2]+random.uniform(-self.var, self.var)]
            # append new particle to the system
            self.children.append( Particle(pos, vel, self.gravity, self.col.copy(), self.colF) )
            

##### Particle
This class has only a constructor and an update function. Particles are completely contained within their parent particle system, and all rendering code is located in the main simulation loop.

This class is identical to that used in the fireworks simulation, except for the last line, which adds a second particle culling criteria

In [4]:
# Definition of particle class
# Parameters:
#    position - [x,y,z]
#    velocity - [x,y,z]
#    acceleration - [x,y,z]
#    startColor - [r,g,b,a]
#    startColor - [r,g,b,a]
#    particles - the number of particles to spwan in explosion
#    lifespan  - the age in seconds at which the particle will die
class Particle:
    def __init__(self, position, velocity, acceleration, startColor, endColor, lifespan = 0.25):
        # particle positional properties
        self.pos = position
        self.vel = velocity
        self.acc = acceleration
        # particle positional properties
        self.col = startColor
        self.colS, self.colF = startColor, endColor
        # life or death properties
        self.age = 0
        self.lifespan = lifespan
        self.alive = True

    def update(self, dt):
        global r
        self.age += dt
        # update velocities
        self.vel[0] += self.acc[0] * dt
        self.vel[1] += self.acc[1] * dt
        self.vel[2] += self.acc[2] * dt
        # update velocities
        self.pos[0] += self.vel[0] * dt
        self.pos[1] += self.vel[1] * dt
        self.pos[2] += self.vel[2] * dt
        # update color
        self.col = colorInterp(self.colS, self.colF, self.age, self.lifespan)
        # kill if too old or below sphere surface
        self.alive = self.age < self.lifespan
        self.alive = self.alive and math.sqrt(self.pos[0]**2+self.pos[1]**2+self.pos[2]**2) > RADIUS


### Helper Functions
Helper functions for color, linear algebra and coordinate frame manipulation.

In [5]:
# COLOR INTERPOLATION
#   interpolated between the intial and final colors as RGB(A?)
#   age is the current age of the particle in frames
#   particle lifespan as the dying age of the particle in frames
#   (lifespan could be optional (=1) if age is a fraction 0.0->1.0)
def colorInterp(initial, final, age, particleLifespan = 1):
    frac = age/particleLifespan
    newR = initial[0] + frac*(final[0]-initial[0])
    newG = initial[1] + frac*(final[1]-initial[1])
    newB = initial[2] + frac*(final[2]-initial[2])
    newA = initial[3] + frac*(final[3]-initial[3])
    return [newR, newG, newB, newA]

# VECTOR NORMALISATION
# accepts a 3D vector and normalises it to be a unit vector.
# returns a copy of the original vector with equal direction but unit magnitude.
def normalise(point):
    mag = math.sqrt(point[0]**2 + point[1]**2 + point[2]**2)
    x = point[0]/mag
    y = point[1]/mag
    z = point[2]/mag
    return [x, y, z]

# SPHERICAL COORDINATES
# Transfers an (x,y,z) position into spherical coordinates
# returns a new vector in spherical coordinates
def xyzToSpherical(xx, yy, zz):
    rr = math.sqrt(xx**2+yy**2+zz**2)
    return [rr, math.acos(zz/rr), math.atan2(yy,xx)]

# Transfers a spherical coordinate into an (x,y,z) position
# returns a new vector in 3d cartesian space
def sphericalToXYZ(rr, pol, azi):
    return [rr*math.sin(pol)*math.cos(azi),
            rr*math.sin(pol)*math.sin(azi),
            rr*math.cos(pol)]

### Simulation Code
The following code defines and implements the main simulation.

##### Definition of contants and window/OpenGL initialisation

In [6]:
# define window constants
WIDTH, HEIGHT, WINDOW_FS = 800, 600, False

# initialise window
if WINDOW_FS:
    win = pyglet.window.Window(fullscreen=True)
    WIDTH, HEIGHT = 1920, 1080
else:
    win = pyglet.window.Window(WIDTH, HEIGHT)

# configure OpenGL to enable transparency
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

In [7]:
# define constants and global variables for simulation
TARGET_FPS = 24
DELTA_T = 1/TARGET_FPS #timestep between frames

systems = [] #collection of all particle subsystems
frameNum = 0 #frame counter

# camera variables
pos = [0, 0, -100] #camera position
rot_deg = 60 # used for the overall rotation and camera move around the planet
rot_vx, rot_vy, rot_vz = 0.0, 1.0, 0.1 # vector about which the camera will rotate

# Planet Radius
rad = RADIUS

##### Generating the sphere (planet surface)
The method used to greate a sphere in pyglet is the 'polygon subdivision and inflation' method described [here](https://stackoverflow.com/questions/7687148/drawing-sphere-in-opengl-without-using-glusphere "stackoverflow"). The implementation of this is not efficient with all points are doubled. Filtering algorithms could potentially used to eliviate this issue, however the effect on performance is currently negigible compared to the particle simulation.

In [8]:
# finds and returns the midpoint of two 3D points, p1 and p2.
def midpoint3f(p1, p2):
    return [(p1[0]+p2[0])/2, (p1[1]+p2[1])/2, (p1[2]+p2[2])/2]

# recursion depth
MAX_DEPTH = 4

# subdivides a triangle into four smaller triangles of half the size.
# triangle is defined by its three corner points, p1, p2 and p3.
# returns a list of vertices for all subdivided triangles
def subdivide(p1, p2, p3, depth=0):
    if depth == MAX_DEPTH:
        # return list of points
        return p1+p2+p3
    else:
        # calculate midpoints
        p12, p13, p23 = midpoint3f(p1, p2), midpoint3f(p1, p3), midpoint3f(p2, p3)
        # use midpoints to recursively subdivide surface and concatenate vertices
        return subdivide(p1, p13, p12, depth+1) + subdivide(p12, p13, p23, depth+1) + subdivide(p12, p23, p2, depth+1) + subdivide(p13, p3, p23, depth+1)


In [9]:
# defining points of octahedron 
px, nx = [rad, 0.0, 0.0], [-rad, 0.0, 0.0] #+/- x-axis
py, ny = [0.0, rad, 0.0], [0.0, -rad, 0.0] #+/- y-axis
pz, nz = [0.0, 0.0, rad], [0.0, 0.0, -rad] #+/- z-axis

# subdivide the surface defined between points in triangles
# triangles form the octahedron
points =  subdivide(py, px, pz)
points += subdivide(py, px, nz)
points += subdivide(py, nx, pz)
points += subdivide(py, nx, nz)
points += subdivide(ny, px, pz)
points += subdivide(ny, px, nz)
points += subdivide(ny, nx, pz)
points += subdivide(ny, nx, nz)

# normalise all points in the points list to have magnitude of planet radius
# has the effect of pillowing/inflating the octahedron into a sphere
for ii in range(0, len(points), 3):
    # calculate vector magnitude
    mag = math.sqrt(points[ii]**2 + points[ii+1]**2 + points[ii+2]**2)
    # scale all points by radius and reciprocal of magnitude
    points[ii]   = rad*points[ii]/mag
    points[ii+1] = rad*points[ii+1]/mag
    points[ii+2] = rad*points[ii+2]/mag
    

##### Main Loop
The main simulation is run in a single loop structure. This function is called automatically by a callback function, and is set to simulate a given global timestep, `DELTA_T`. This function is responsible for updating particle systems and their particles. All drawing code is contained within this loop, and contributes significantly to the performance of the program.

In each frame, the loop operates in the following order:
1. Establish OpenGL projection matrix and camera settings
1. Translate and rotate the camera into position
1. Draw the sphere
1. Update the particle system
1. Draw particles

Note that the rear clipping plane of the camera has been placed in the middle of the planet to simulate a fragment shaders clipping.

In [10]:
def mainLoop(dt):
    global pos_z, rot_deg, points
    win.clear() # clear screen for redrawing
    # set up 3D view of space
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(90, WIDTH/HEIGHT, 0.1, 100) #set FoV and clipping planes
    # translate/rotate camera into desired position
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glTranslatef(*pos)
    glRotatef(rot_deg, rot_vx, rot_vy, rot_vz)

    #-----------------------------
    # draw sphere
    # block below is for drawing the vertices creating the sphere
    # glBegin(GL_POINTS)
    # glColor4f(1.0, 1.0, 1.0, 0.8)
    # for i in range(0, len(points), 3):
    #     glVertex3f(points[i], points[i+1], points[i+2])
    # glEnd()
    
    # block below is for drawing faces on to the sphere
    glBegin(GL_TRIANGLES)
    glColor4f(1.0, 1.0, 1.0, 0.05)
    for ii in range(0, len(points), 3):
        glVertex3f(points[ii], points[ii+1], points[ii+2])
    glEnd()
    rot_deg -= 0 # the number of degrees by which to rotate the camera over each frame

    #-----------------------------
    # draw particles
    global systems, frameNum
    
    # update particle system
    particles = []
    for index, sys in reversed(list(enumerate(systems))):
        sys.update(DELTA_T) #updates the particle system, which triggers an update of its children
        if sys.alive:
            # add particles to list
            particles += sys.children 
        else:
            # delete particle system
            del systems[index]

    # draw pixels -> may be able to remove one loop with reordering and optimisation
    glBegin(GL_POINTS)
    for pp in particles:
        glColor4f(pp.col[0], pp.col[1], pp.col[2], pp.col[3]) # define point color
        glVertex3f(pp.pos[0], pp.pos[1], pp.pos[2]) # draws point at location
    glEnd()

    frameNum += 1
    
    # saves the current frame into the directory ./frames/<frame number>.png <- this directory may need to be created
    # pyglet.image.get_buffer_manager().get_color_buffer().save('frames/' + str(frameNum)+'.png')
    
    # status logging - repeatedly prints to terminal. NOT SUITABLE FOR PYTHON NOTEBOOOK
    # print( "Frame %3d: %6d particles, %3d systems, %4d ms (%2d FPS)" % (frameNum, len(particles), len(systems), dt, 1/dt) )
    
    glFlush() # flushes the OpenGL pipeline


##### Top Level Particle System
This function acts as the top level particle system, and spawns particle systems into the simulation. It has been set as a callback function in order to periodically repeat the simulation, however other more complex behaviour can be achieved by customising this function and its calling method.

The function spawns particle systems in concentric rings around the sphere, centered along the x-axis. This progressed down the length of the sphere, creating the illusion of a wave of fire engulfing the planet.

In [11]:
# constant values for top level particle system function
iteration = 0 # counts how far through sphere spread simulation is
STEPS = 100

diameter = np.linspace(-rad, rad, STEPS) # array of x values across the center of the sphere (points along the x axis)
DENSITY = 0.9 # density of particle system spawning
LIFE_MEAN, LIFE_VAR = 2, 1 #mean and variance of particle system lifetimes

# Callback function for spawning particles
# parameter dt is automatically given the value of the elapsed time intervall by the calling function
# moves through the sphere of the planet along the x-axis and spwans particles around the disk centered at that x coordinate
def spawn(dt):
    global rad, iteration
    if iteration < STEPS:
        # get x position and radius of disk at x location
        xval = diameter[iteration]
        minRad = math.sqrt( rad**2 - xval**2 )
        
        # spawn a series of particle systems along disk (number depends on disk radius)
        for ii in range(round(DENSITY*minRad)):
            ang = random.uniform(0, 2*math.pi) # angle around disk (polar coordinates)
            life = LIFE_MEAN + random.uniform(-LIFE_VAR, LIFE_VAR) #lifespan or particle
            #append new particle system to simulation
            systems.append( ParticleSystem([xval, minRad*math.sin(ang), minRad*math.cos(ang)],
                                           [1.0, 0.5, 0.05, 0.95],
                                           [1.0, 0.0, 0.1, 0.1],
                                           lifespan=life) )

    iteration = (iteration+1)%200 #move iteration from 0 to 200 (but only spawn systems from 0 to STEPS)


### Running the Simulation
The following code sets timers for the main simulation loop and the particle system spawning function.
The app is then launched in a separate window.

**NOTE THAT PERFORMANCE IS VERY VERY POOR**

Once the simulation window has been closed and the simulation stopped, **the kernel may need to be restarted** before the simulation can run again.

In [12]:
# sets clock for callbacks
pyglet.clock.schedule_interval(mainLoop, DELTA_T)
pyglet.clock.schedule_interval(spawn, 0.25)

# launches simultation
pyglet.app.run()

### Improvements
There are many improvements that can be made to the code. Here are some of them:
1. Better rendering techniques and compositing. The use of a fragment shader, as well as improved camera movement and particle clipping would allow for a much more impressive result, includding techniques such as motion blur.
1. The initial velocities and accelerations of the particles are framerate dependant. Altering the target framerate will change the overall motion of the simulation.
1. Performance improvents can be made to the rendering of the sphere as well as the data structures and algorithms implemented. Experimentation regarding the performance boost, quality loss, and memory increase of moving away from random number generation in favour of a look up scheme could also be investigated.